In [ ]:
from transformers import AutoModel, AutoTokenizer
import torch
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
model_name = 'deepseek-ai/DeepSeek-OCR'

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(model_name, _attn_implementation='flash_attention_2', trust_remote_code=True, use_safetensors=True)
model = model.eval().cuda().to(torch.bfloat16)

In [6]:
model.device

device(type='cuda', index=0)

In [11]:
# prompt = "<image>\nFree OCR. "
prompt = "<image>\n<|grounding|>Convert the document to markdown. "
image_file = 'data/airbus_table.png'
output_path = 'data/output'

# infer(self, tokenizer, prompt='', image_file='', output_path = ' ', base_size = 1024, image_size = 640, crop_mode = True, test_compress = False, save_results = False):

# Tiny: base_size = 512, image_size = 512, crop_mode = False
# Small: base_size = 640, image_size = 640, crop_mode = False
# Base: base_size = 1024, image_size = 1024, crop_mode = False
# Large: base_size = 1280, image_size = 1280, crop_mode = False

# Gundam: base_size = 1024, image_size = 640, crop_mode = True

res = model.infer(tokenizer, prompt=prompt, image_file=image_file, output_path = output_path, base_size = 1280, image_size = 1280, crop_mode=False, save_results = True, test_compress = True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


BASE:  torch.Size([1, 400, 1280])
NO PATCHES
<|ref|>table<|/ref|><|det|>[[0, 0, 999, 999]]<|/det|>
<table><tr><td>Granted Date</td><td>LTIP 2019</td><td>LTIP 2020</td><td>LTIP 2021</td><td>LTIP 2022</td><td>LTIP 2023</td><td>LTIP 2024</td></tr><tr><td>Performance Units and Shares</td><td>11,060</td><td>19,840</td><td>12,121</td><td>14,115</td><td>11,619</td><td>10,839</td></tr><tr><td>Revaluation</td><td>50%</td><td>150%</td><td>96%</td><td>100%</td><td>100%</td><td>100%</td></tr><tr><td>Performance Units and Shares revalued</td><td>5,531</td><td>29,760</td><td>11,636</td><td>14,115</td><td>11,619</td><td>10,839</td></tr><tr><td>Vested in 2024</td><td></td><td></td><td></td><td></td><td></td><td></td></tr><tr><td>in cash</td><td>1,383</td><td>7,440</td><td>-</td><td>-</td><td>-</td><td>-</td></tr><tr><td>in shares</td><td>-</td><td>14,880</td><td>-</td><td>-</td><td>-</td><td>-</td></tr><tr><td>Outstanding 2024</td><td></td><td></td><td></td><td></td><td></td><td></td></tr><tr><td>In c

image: 0it [00:00, ?it/s]
other: 100%|██████████| 1/1 [00:00<00:00, 35246.25it/s]


In [7]:
res2 = model.infer(tokenizer, prompt=prompt, image_file=image_file, output_path = output_path, base_size = 512, image_size = 512, crop_mode=False, save_results = True, test_compress = True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


directly resize
BASE:  torch.Size([1, 64, 1280])
NO PATCHES
<|ref|>sub_title<|/ref|><|det|>[[35, 160, 260, 174]]<|/det|>
## Post-Employment Benefit Costs 

<|ref|>text<|/ref|><|det|>[[35, 177, 470, 217]]<|/det|>
Post-employment benefit costs relate to the aggregated amount of current service units costs for defined benefit plan and company cost for contributions base plans. 

<|ref|>text<|/ref|><|det|>[[35, 230, 470, 270]]<|/det|>
Following the Board decision approving the A&M 2003, the CEO pension committee voted unanimously a defined contribution plan (summary plan), which coexists with the former defined benefit pension plan. 

<|ref|>text<|/ref|><|det|>[[35, 283, 470, 336]]<|/det|>
The accrued expense under the former defined benefit plan base plan was the end of 2009 and represents unrealized net the retirement date of the CEO. The pension plan adopted the Company's defined contribution plan are described from the treasury defined pension register. 

<|ref|>text<|/ref|><|det|>[[35

image: 0it [00:00, ?it/s]
other: 100%|██████████| 19/19 [00:00<00:00, 176309.24it/s]
